### Exploring The Implementation of KS-DFT

#### References
Theory: <br>
https://www.southampton.ac.uk/assets/centresresearch/documents/compchem/DFT_L7.pdf <br>
https://github.com/slipchenko/CHM673/tree/master/lectures <br>
Jensen Chapter 6

Implementation: <br>
https://github.com/psi4/psi4numpy/tree/master/Tutorials/03_Hartree-Fock <br>
https://github.com/psi4/psi4numpy/tree/master/Tutorials/04_Density_Functional_Theory <br>
particularly 3a,3b,4a,4b

Last week we wrapped up our discussion on HF by finishing our derivation of the HF and Roothaan equations, briefly looked at the computational implementation of HF, and introduced basis sets. This week the tutorial will focus on KS-DFT, starting from the KS Energy functional, with the goal of learning how KS-DFT is computationally implemented, while drawing comparisons to our HF discussion.

### Kohn-Sham Equations

Traditionally we write the KS Energy in the following way: <br>
$$ E_{KS}[n]=T_{S}[n]+E_{ext}+E_H + E_{XC}               $$ <br>
We can rewrite this using slightly different notation: <br>
$$ E_{KS}[n]=T_{S}[n]+E_{Ne}+J + E_{XC} $$ <br>
And compare to the HF Energy we saw last week,
$$ E_{HF}=T+E_{Ne}+J-K $$
We see that from this perspective, HF and KS-DFT are very similiar. And in fact, because we take this orbital based approach, they are even more similiar: Both HF and KS-DFT can be seen as minimizing their respective energy functionals with the constraint that their respetive orbitals remain orthonormal. Moreover, we in fact solve this minimization problem the same way we did last week for HF, using Lagrangian multipliers. 

First we write our Lagrangian:
$$ \Omega_{KS}[n]=E_{KS}[n]-2\sum_{i=1}^{N_{elec}/2} \sum_{j=1}^{N_{elec}/2} \epsilon_{ij} \big(\int \phi_i^*(r)\phi_j(r)dr-\delta_{ij}\big) $$ <br>
and vary our Lagrangian wrt. the orbitals st. we find the stationary point:
$$  \frac{\delta\Omega_{KS}[n]}{\delta\phi_j^*(r)} = 0   $$
Remember, the goal is to find the set of orthogonal orbitals which yield the lowest energy

And using the chain rule: <br>
$$\frac{\delta\Omega_{KS}[n]}{\delta\phi_j^*(r)} = \frac{\delta\Omega_{KS}[n]}{\delta n(r)} \frac{\delta n(r)}{\delta\phi_j^*(r)}   $$

$$\frac{\delta n(r)}{\delta\phi_j^*(r)} = 2 \phi_j$$

Ultimately we are left with the following 1 electron schrodinger equations, known as the Kohn-Sham Equations:
$$ \big(\frac{-1}{2}\nabla^2 +\big(\frac{\delta E_{coul}}{\delta n} + \frac{\delta E_{ext}}{\delta n} + \frac{\delta E_{XC}}{\delta n}   \big) \big)\psi_j(r) = \epsilon_j \psi_j(r)  $$

$$  \big( \frac{-1}{2}\nabla^2 +\nu_s \big)\psi_j(r) = \epsilon_j\psi_j(r)   $$
Where $\psi$ is the unitarily transformed $\phi$ s.t. the electron density is identical, but the matrix of lagrange multipliers is diagonal.

Analogous to HF we define a 1 electron operator <br>

$\hat{h}_{KS}= \frac{-1}{2}\nabla^2+\nu_s$

$ \hat{h}_{KS}\psi_j(r) = \epsilon_j(r)\psi_j(r) $

This is again a pseduo-eigenvalue problem, and we solve it almost identically to the HF method, in which we make an inital guess at our orbitals, solve the KS-equations for new orbitals, and repeat until self-consistency. The details of which we will now outline.

### Implementation

We will pick some basis set $\{\chi_\mu \}$to represent our KS orbitals $\psi$
$$\psi_i = \sum_\mu^{K}c_{\mu i}\chi_\mu$$

We can then rewrite our KS Equations into a matrix equation:
$$h_{KS}C = SC\epsilon   $$
This is nearly identical to the Roothan equations, in fact our $h_{KS}$ has identical 1-electron and coulomb components, and differs only in the exchange-correlation component, whose matrix components are (in general) given by:

$$\int \chi_\mu(r)V_{XC}\big[n(r),\nabla n(r)\big]\chi_\nu(r) dr  $$

Which must be evaluated numerically on a grid with G points:

$$\sum_k^G V_{XC}\big[n(r_k),\nabla n(r_k)\big]\chi_\mu(r_k) \chi_\nu(r_k) \Delta v_k $$

We wont get into as much detail as last week in our implementation of the DFT SCF method, many of the details are analagous to the HF SCF cycle. Instead, I hope to outline the key components of the SCF cycle, allowing PySCF/libxc to handle the more complicated aspects.

#### Lets try!

The general scheme will be similiar to how the HF SCF scheme was implemented, but we define a different Fock operator, using the LDA Exchange-Correlation potential instead of the exact exchange.

In [1]:
from pyscf import gto, dft, scf
import numpy as np

/depot/awasser/apps/psi4_1.8_pyscf_2.3.0/lib/python3.8/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [8]:
mol = gto.M(atom =f"""
    O 1 
    H 1 0.74
    H 1 0.74 2 104
""", basis = 'sto-3g', spin=0, charge=0, verbose=3)
#Here we use the sto-3g basis for simplicity4

# Let us do a normal PySCF DFT calculation for reference
mydft = dft.RKS(mol)
mydft.xc = 'lda'
mydft.kernel()
D_ref = mydft.make_rdm1()

#Let us define the following integrals using pyscf
T = mol.intor('int1e_kin') 
V = mol.intor('int1e_nuc') 
S = mol.intor('int1e_ovlp')
I = mol.intor('int2e')

#our core hamiltonian
H=T+V

#we need to construct the matrix that diagonalizes S
e, U = np.linalg.eigh(S)
A = U @ np.diag(e**-0.5) @ U.T

#We are doing a restricted calculation, so each orbital contains 2 electrons
ndocc= mol.nelectron // 2

converged SCF energy = -73.7996544212561


In [5]:
# Initial guess at D using just H_core 

F_p = A.dot(H).dot(A)
# Diagonalize F_p for eigenvalues & eigenvectors with NumPy
e, C_p = np.linalg.eigh(F_p)
# Transform C_p back into AO basis
C = A.dot(C_p)
# Grab occupied orbitals
C_occ = C[:, :ndocc]
# Build density matrix from occupied orbitals
D = 2 * np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)

In fact, the naive SCF procedure can be quite poor at converging, and there have been several methods developed to help iterative solutions converge faster, one of the most ubiqutious being Direct Inversion of the Iterative Subspace (DIIS). DIIS is the default method for scf calculations in PySCF, and the SCF calculation below struggles to converge without some accelerated convergence method. DIIS might be the topic of a future tutorial, so dont get too focused on the details of it here. If you are interested, check out psi4 tutorial 3b, and https://en.wikipedia.org/wiki/DIIS.

In [9]:
# Begin SCF Iterations
print('==> Starting SCF Iterations <==\n')
E_old = 0.0

#this is for DIIS
# Trial & Residual Vector Lists
F_list = []
DIIS_RESID = []


# Maximum SCF iterations
MAXITER = 40
# Energy convergence criterion
E_conv = 1.0e-8
D_conv = 1.0e-6

for scf_iter in range(1, MAXITER + 1):
    # Build Fock matrix
    #print(D)
    J = np.einsum('pqrs,rs->pq', I, D, optimize=True)
    ni = dft.numint.NumInt()
    n, exc, vxc = ni.nr_rks(mol, mydft.grids, 'lda', D)
    F = H + J + vxc

    H_E = np.sum(D * H)
    J_E = 0.5 * np.sum(D * J)
    
    # Build DIIS Residual
    diis_r = A.dot(F.dot(D).dot(S) - S.dot(D).dot(F)).dot(A)
    # Append trial & residual vectors to lists
    F_list.append(F)
    DIIS_RESID.append(diis_r)
    
    
    SCF_E = H_E + J_E + exc + mol.energy_nuc()
    dE = SCF_E - E_old
    dRMS = np.mean(diis_r**2)**0.5
    print('SCF Iteration %3d: Energy = %4.16f dE = % 1.5E dRMS = %1.5E' % (scf_iter, SCF_E, dE, dRMS))

    # Check for convergence
    if (abs(dE) < E_conv) and (dRMS < D_conv):
        break
    E_old = SCF_E
    
    #DIIS procedure
    if scf_iter >= 2:
        # Build B matrix
        B_dim = len(F_list) + 1
        B = np.empty((B_dim, B_dim))
        B[-1, :] = -1
        B[:, -1] = -1
        B[-1, -1] = 0
        for i in range(len(F_list)):
            for j in range(len(F_list)):
                B[i, j] = np.einsum('ij,ij->', DIIS_RESID[i], DIIS_RESID[j], optimize=True)

        # Build RHS of Pulay equation 
        rhs = np.zeros((B_dim))
        rhs[-1] = -1
        
        # Solve Pulay equation for c_i's with NumPy
        coeff = np.linalg.solve(B, rhs)
        
        # Build DIIS Fock matrix
        F = np.zeros_like(F)
        for x in range(coeff.shape[0] - 1):
            F += coeff[x] * F_list[x]
    

    
    # Compute new orbital guess
    F_p = A.dot(F).dot(A)
    e, C_p = np.linalg.eigh(F_p)
    C = A.dot(C_p)
    C_occ = C[:, :ndocc]
    D = 2* np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)

# Post iterations
print('\nSCF converged.')
print(f'Final RHF Energy: {SCF_E:.13f} [Eh]')

==> Starting SCF Iterations <==

SCF Iteration   1: Energy = -73.7996544212542176 dE = -7.37997E+01 dRMS = 5.51968E-07
SCF Iteration   2: Energy = -73.7996544212555676 dE = -1.35003E-12 dRMS = 3.11610E-07

SCF converged.
Final RHF Energy: -73.7996544212556 [Eh]


### Homework
* Why do we need to solve the KS equations self consistently?


* Effect of the basis set
    * How does changing the size of basis set change our energy (higher/lower?)
    * How many basis functions does H2O have with STO-3G. How many primitive gaussians?
    * What are the dimensions of our Fock matrix?
        * how does changing the basis set affect the Fock matrix
    * does changing the basis set affect convergence?
        * does it affect the speed of each iteration?
    * What is the limiting factor in the scope of an SCF calculation?
    
    
* Effect of the inital guess
    * how does the inital guess for D affect convergence? try different inital guesses and see how they converge
    
    
    
* Last week we did Geometry Optimization of the HeH+ dimer, can you do the same geometry optimzation, this time using DFT? What are the differences?


* Think of a situation where our SCF procedure might fail, Can you set up a calculation that fails to converge? feel free to modify the molecule, geometry, initial guess, and convergence criteria.


* can you extract the molecular orbital energies? 
    * What does the number of molecular orbitals depend on?
        * How many molecular orbitals do we have for H2O with STO-3G? 
            * How many are negative in energy? 
        * How many molecular orbitals do we have for H2O with aug-cc-pvdz? 
            * How many are negative in energy? 
* Can you Create a Z-matrix for H3O+?

    mol = gto.M(atom =f"""
    O 1 
    H 1 0.74
    H 1 0.74 2 104
    H 1 0.74 2 104 ?
    """, basis = 'sto-3g', spin=0, charge=1, verbose=3)'
    
* can you run an H3O+ geometry calculation?